# Installing packages

In [1]:
!pip install -q openai --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


# Importing modules

In [2]:
from openai import OpenAI
import time
import pandas as pd
import os
import json
from google.colab import drive
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="API_KEY",
)

# Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive.mount('/content/drive')

In [7]:
# primate_input_df = pd.read_csv('/content/drive/MyDrive/primate_data.csv')
primate_input_df = pd.read_csv('primate_data.csv')

In [5]:
primate_input_df.head()

,post_title,post_text,Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down,Feeling-down-depressed-or-hopeless,Feeling-tired-or-having-little-energy,Little-interest-or-pleasure-in-doing,Moving-or-speaking-so-slowly-that-other-people-could-have-noticed-Or-the-opposite-being-so-fidgety-or-restless-that-you-have-been-moving-around-a-lot-more-than-usual,Poor-appetite-or-overeating,Thoughts-that-you-would-be-better-off-dead-or-of-hurting-yourself-in-some-way,Trouble-concentrating-on-things-such-as-reading-the-newspaper-or-watching-television,Trouble-falling-or-staying-asleep-or-sleeping-too-much
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",1,0.0,1,1,0,0.0,0.0,0.0,0
1,I don’t have anyone to talk to and I don’t kno...,Nine years ago I was diagnosed with depression...,1,1.0,0,0,1,0.0,0.0,0.0,0
2,Stuck between moving forward and killing myself,Some background information: My GF of almost 3...,1,1.0,1,1,0,0.0,1.0,0.0,0
3,I need help on how to help my girlfriend.,"My girlfriend ,of about 3 months now ,has been...",1,1.0,0,0,0,0.0,1.0,0.0,0
4,I've been feeling this way for some tine,I'm alway feeling like this. It doesn't even m...,1,1.0,0,0,0,0.0,0.0,0.0,0


# Task

In [4]:
pre_processed_labels = {
    "Q1": "Feeling bad about yourself or that you are a failure or have let yourself or your family down",
    "Q2": "Feeling down depressed or hopeless",
    "Q3": "Feeling tired or having little energy",
    "Q4": "Little interest or pleasure in doing",
    "Q5": "Moving or speaking so slowly that other people could have noticed Or the opposite being so fidgety or restless that you have been moving around a lot more than usual",
    "Q6": "Poor appetite or overeating",
    "Q7": "Thoughts that you would be better off dead or of hurting yourself in some way",
    "Q8": "Trouble concentrating on things such as reading the newspaper or watching television",
    "Q9": "Trouble falling or staying asleep or sleeping too much"

}

In [5]:
def get_answer_2023_11_28(post_title, post_text, label):
  try:
    messages = [
                {"role": "system", "content": "You are a helpful assistant in the field of mental health."},
                {"role": "user", "content": "Hello, assistant."},
                {"role": "assistant", "content": "Hello! How can I assist you today?"},
                {"role": "user", "content": "Classify this post for the given post label. Your answer must be only a JSON object with no further explanations."}
              ]
    intro_message = f"""
    This is a text classification task.
    I have collected a dataset for my machine learning task which is related to mental health issues.
    I have found the PRIMATE dataset which was collected in 2022.
    The dataset contains 2003 records, of which every record has the below structure:
      post_title - post_text - Q1 - Q2 - Q3 - Q4 - Q5 - Q6 - Q7 - Q8 - Q9
      The post_titles, and post_texts were collected from the Reddit website,
      and the Q1 to Q9 were some matrices based on the PHQ-9(Patient Health Questionnaire ).
      The value of the Q1 to Q9 questions can be deduced/known using the content present in the post text and its title.
      I will give the post title with its post text and a label which are delimited by triple backticks.
      But I want from you to tell me whether the given label(question) would be yes or no by your own inferring.

      First, read the input post text, post title and the given label.
      Second, deduce that whether the problem which is mentioned in label exists in the post content or not, and save your REASON(Limit the number of words to 100).
      Third, based on your reason and inferring determine whether the given label(question) would be yes or no for the given inputs.
      If the post text or post title contains some ilegal problems please ignore them as you can.
      Since I am just trying to compare your outputs with the output of downloaded dataset.
      Provide your answer for the given data in JSON format with the following keys:
      reason(your REASON from the second step), value(yes/no), post_title, label.

      This is the given post title: ```{post_title}```, this is the post text: ```{post_text}```, and this is the label: ```{label}```.
    """
    messages.append( {"role": "user", "content": intro_message}, )
    chat = client.chat.completions.create( model="gpt-3.5-turbo-0613", messages=messages )

    reply = chat.choices[0].message.content
    return True, reply

  except Exception as e:
    return False, e

## 50 to 60

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
sixth_ten_df = pd.DataFrame(columns=columns)

In [ ]:
sixth_ten_df.head()

,post_title,label,value


In [ ]:
# sixth_ten_rows =  primate_input_df.iloc[50:60]
sixth_ten_rows =  primate_input_df.iloc[56:60]

total_rows = len(sixth_ten_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in sixth_ten_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            sixth_ten_df = sixth_ten_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 7 minutes interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    sixth_ten_df = sixth_ten_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      # Add to dataframe
      # Create a new JSON file for each iteration
      file_path = f"gpt_result_{post_title}_{key}.json"  # Customize the file name as desired

      with open(file_path, "w") as json_file:
          json.dump(result, json_file, indent=2)

      progress_bar.update(1)


Processing Rows:   0%|          | 0/36 [06:21<?, ?it/s]
<ipython-input-26-eaf033a146d9>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixth_ten_df = sixth_ten_df.append({

Processing Rows:   3%|▎         | 1/36 [00:56<33:10, 56.87s/it]

The result is: True and {
  "post_title": "I have been severely depressed and it is affecting my school work and looking for advice",
  "post_text": "Since I've hit my 20's, my depression has just plummeted. In high school, I hadn't been diagnosed yet with my anxiety and depression, and even though it was present it didn't feel nearly as bad as this. I hit my 20's, now 23 yrs old, and I am having constant depression cycles. The cycles are becoming more frequent and each time it feels harder to come out of. As of lately I have been probably the most down that I have felt to date. I am in my sophomore year of college as a bio major. I haven't been keeping up with classes, I have been failing exams and I don't have nearly enough credits to keep my scholarship for my junior year. I have fucked myself so badly with school I don't know what to do. I want so badly to take a leave of absence to get myself together, up my medication and see a therapist. I really want to get better. However, if 

<ipython-input-26-eaf033a146d9>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixth_ten_df = sixth_ten_df.append({

Processing Rows:   6%|▌         | 2/36 [01:07<16:46, 29.59s/it]

The result is: True and {
  "reason": "The post text indicates that the person is experiencing severe depression and it is negatively impacting their school work, which aligns with the given label 'Feeling down depressed or hopeless'.",
  "value": "Yes",
  "post_title": "I have been severely depressed and it is affecting my school work and looking for advice",
  "label": "Feeling down depressed or hopeless"
}
{'reason': "The post text indicates that the person is experiencing severe depression and it is negatively impacting their school work, which aligns with the given label 'Feeling down depressed or hopeless'.", 'value': 'Yes', 'post_title': 'I have been severely depressed and it is affecting my school work and looking for advice', 'label': 'Feeling down depressed or hopeless'}


<ipython-input-26-eaf033a146d9>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixth_ten_df = sixth_ten_df.append({

Processing Rows:   8%|▊         | 3/36 [01:14<10:35, 19.26s/it]

...

The result is: True and {
  "reason": "The post text mentions that the friend has had depression for at least 4 years, has attempted suicide and self-harm, and expresses uncertainty about the friend's ability to hold on. These thoughts align with the label/question of 'Thoughts that you would be better off dead or of hurting yourself in some way.'",
  "value": "yes",
  "post_title": "My friend is dying inside and I don't know how to help him",
  "label": "Thoughts that you would be better off dead or of hurting yourself in some way"
}
{'reason': "The post text mentions that the friend has had depression for at least 4 years, has attempted suicide and self-harm, and expresses uncertainty about the friend's ability to hold on. These thoughts align with the label/question of 'Thoughts that you would be better off dead or of hurting yourself in some way.'", 'value': 'yes', 'post_title': "My friend is dying inside and I don't know how to help him", 'label': 'Thoughts that you would be bette

<ipython-input-26-eaf033a146d9>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixth_ten_df = sixth_ten_df.append({

Processing Rows:  97%|█████████▋| 35/36 [09:38<00:10, 10.37s/it]

The result is: True and {
  "reason": "The given label/question is unrelated to the content of the post text and post title.",
  "value": "no",
  "post_title": "My friend is dying inside and I don't know how to help him",
  "label": "Trouble concentrating on things such as reading the newspaper or watching television"
}
{'reason': 'The given label/question is unrelated to the content of the post text and post title.', 'value': 'no', 'post_title': "My friend is dying inside and I don't know how to help him", 'label': 'Trouble concentrating on things such as reading the newspaper or watching television'}
The result is: True and {
  "post_title": "My friend is dying inside and I don't know how to help him",
  "post_text": "My friend has had depression for at least 4 years now. My friends and I have been helping him through his depression for a while. He's been through therapy and had been admitted to a mental hospital about a year ago for attempted suicide and self-harm. But it doesn't fe

<ipython-input-26-eaf033a146d9>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixth_ten_df = sixth_ten_df.append({

Processing Rows: 100%|██████████| 36/36 [11:54<00:00, 48.30s/it]

{'reason': 'The post text indicates that the person is experiencing depression symptoms, such as feeling numb and lacking motivation. However, it does not specifically mention trouble falling or staying asleep or sleeping too much.', 'value': 'no', 'post_title': "My friend is dying inside and I don't know how to help him", 'label': 'Trouble falling or staying asleep or sleeping too much'}
attempt 3 was SUCCESSFUL


In [ ]:
sixth_ten_df.to_csv('sixth_ten_data.csv', index=False)

## 60 to 80

In this scenario it takes more time since more cases had been failed

The 60 is duplicate

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
sixty_to_eighty_df = pd.DataFrame(columns=columns)

In [ ]:
sixty_to_eighty_rows =  primate_input_df.iloc[60:80]

total_rows = len(sixty_to_eighty_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in sixty_to_eighty_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            sixty_to_eighty_df = sixty_to_eighty_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 7 minutes interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    sixty_to_eighty_df = sixty_to_eighty_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      progress_bar.update(1)

Processing Rows: 100%|██████████| 36/36 [23:13<00:00, 38.70s/it]


The result is: True and {
  "reason": "The post text describes a feeling of apathy and lack of motivation, but does not mention feeling bad about oneself or being a failure. Therefore, the given label/question would be 'no' for the given inputs.",
  "value": "no",
  "post_title": "What activities do you like to do to face your apathy?",
  "label": "Feeling bad about yourself or that you are a failure or have let yourself or your family down"
}
{'reason': "The post text describes a feeling of apathy and lack of motivation, but does not mention feeling bad about oneself or being a failure. Therefore, the given label/question would be 'no' for the given inputs.", 'value': 'no', 'post_title': 'What activities do you like to do to face your apathy?', 'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down'}


<ipython-input-32-cb3745089fdd>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixty_to_eighty_df = sixty_to_eighty_df.append({


The result is: True and {
  "reason": "The post text mentions feeling apathetic, lacking motivation, and not experiencing any emotions. These symptoms align with feeling down and depressed.",
  "value": "yes",
  "post_title": "What activities do you like to do to face your apathy?",
  "label": "Feeling down depressed or hopeless"
}
{'reason': 'The post text mentions feeling apathetic, lacking motivation, and not experiencing any emotions. These symptoms align with feeling down and depressed.', 'value': 'yes', 'post_title': 'What activities do you like to do to face your apathy?', 'label': 'Feeling down depressed or hopeless'}


<ipython-input-32-cb3745089fdd>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixty_to_eighty_df = sixty_to_eighty_df.append({


...

<ipython-input-32-cb3745089fdd>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixty_to_eighty_df = sixty_to_eighty_df.append({


The result is: True and {
  "reason": "The given post content does not mention any trouble with concentration on activities such as reading or watching television.",
  "value": "no",
  "post_title": "Spiraled into depression what do I do :(",
  "label": "Trouble concentrating on things such as reading the newspaper or watching television"
}
{'reason': 'The given post content does not mention any trouble with concentration on activities such as reading or watching television.', 'value': 'no', 'post_title': 'Spiraled into depression what do I do :(', 'label': 'Trouble concentrating on things such as reading the newspaper or watching television'}


<ipython-input-32-cb3745089fdd>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixty_to_eighty_df = sixty_to_eighty_df.append({


The result is: True and {
  "post_title": "Spiraled into depression what do I do :(",
  "post_text": "I'm still in high school and I've missed a ton of school days because of my depression. My family knows about this but there isn't much they can do to help. I'm trying meds they're not working and my family wont allow me to go to a therapist. I've also talked to my school counselor but she has not met with me in weeks so I can't talk to her!! I'm so stuck I'm so scared and I don't know what to do anymore. I can't take it I feel like no one understands me and I don't know how to bring myself back up on my feet again, I'm just so exhausted I feel like giving up.",
  "label": "Trouble falling or staying asleep or sleeping too much",
  "reason": "The post text provides information about the person's depression, feelings of being stuck and scared, exhaustion, and a mention of not knowing how to bring themselves back up. Though it does not specifically mention trouble falling or staying asle

<ipython-input-32-cb3745089fdd>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sixty_to_eighty_df = sixty_to_eighty_df.append({


In [ ]:
sixty_to_eighty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  179 non-null    object
 1   label       179 non-null    object
 2   value       179 non-null    object
 3   reason      179 non-null    object
dtypes: object(4)
memory usage: 5.7+ KB


In [ ]:
sixty_to_eighty_df.to_csv('sixty_to_eighty_data.csv', index=False)

## 80 to 90

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
eighty_to_ninty_df = pd.DataFrame(columns=columns)

In [ ]:
eighty_to_ninty_rows =  primate_input_df.iloc[80:90]

total_rows = len(eighty_to_ninty_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in eighty_to_ninty_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            eighty_to_ninty_df = eighty_to_ninty_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 7 minutes interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    eighty_to_ninty_df = eighty_to_ninty_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      progress_bar.update(1)

Processing Rows:   0%|          | 0/90 [00:09<?, ?it/s]
<ipython-input-37-f7925241e3e7>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eighty_to_ninty_df = eighty_to_ninty_df.append({
Processing Rows:   1%|          | 1/90 [00:07<11:35,  7.82s/it]

The result is: True and {
  "reason": "The post text reflects the person feeling bad about themselves and expressing a desire for change.",
  "value": "yes",
  "post_title": "I am a toxic person and I want to not be like this anymore.",
  "label": "Feeling bad about yourself or that you are a failure or have let yourself or your family down"
}
{'reason': 'The post text reflects the person feeling bad about themselves and expressing a desire for change.', 'value': 'yes', 'post_title': 'I am a toxic person and I want to not be like this anymore.', 'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down'}


<ipython-input-37-f7925241e3e7>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eighty_to_ninty_df = eighty_to_ninty_df.append({
Processing Rows:   2%|▏         | 2/90 [00:27<21:25, 14.61s/it]

The result is: True and {
  "post_title": "I am a toxic person and I want to not be like this anymore.",
  "post_text": "I am a 27 year old man who has recently come to the conclusion that he is toxic. Here's why...\n\nI never take to the internet in search for solutions, but if anyone knows how to change their attitude so that they may continue on the pursuit for happiness, then please, I beg of you, share your wisdom. I don't want to be this person.",
  "label": "Feeling down depressed or hopeless",
  "reason": "The post text indicates that the person is expressing feelings of self-pity, emotional outrage and despair. They mention they have lost passion, feel lonely, and have no confidence in their social abilities. These feelings align with the label 'Feeling down depressed or hopeless'.",
  "value": "yes"
}
{'post_title': 'I am a toxic person and I want to not be like this anymore.', 'post_text': "I am a 27 year old man who has recently come to the conclusion that he is toxic. Here

<ipython-input-37-f7925241e3e7>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eighty_to_ninty_df = eighty_to_ninty_df.append({
Processing Rows:   3%|▎         | 3/90 [00:36<17:23, 11.99s/it]

...

The result is: True and {
  "post_title": "What is \"you\" time?",
  "post_text": "Hey everyone. So I've been depressed for a large portion of my life. Almost 20 years now..wow. That's really hard to actually type that. But it is what it is... I am working towards getting better. I have no health insurance so doctors, meds, and therapy are not options to me right now. But I am trying to develop habits that I consider helpful. Researching as much as I can and including that advice into my own treatment.\n\nOf course, when I can afford it, I do plan on getting therapy and the works. But this is the best I can do at the moment.\n\nOne thing I see a lot is to make time for \"you\". Make time for \"yourself\". I have been struggling for months to figure out what this really means. Is it being productive? Or is it just being relaxed and leisurely? Is it a moment of quiet? Is it doing some sort of hobby? I wanted to see if anyone had ideas of what it would mean.\n\nI have a feeling that it ca

<ipython-input-37-f7925241e3e7>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eighty_to_ninty_df = eighty_to_ninty_df.append({
Processing Rows:  99%|█████████▉| 89/90 [46:52<00:25, 25.20s/it]

The result is: True and {
  "reason": "The given label is unrelated to the content of the post text and post title.",
  "value": "no",
  "post_title": "What is \"you\" time?",
  "label": "Trouble concentrating on things such as reading the newspaper or watching television"
}
{'reason': 'The given label is unrelated to the content of the post text and post title.', 'value': 'no', 'post_title': 'What is "you" time?', 'label': 'Trouble concentrating on things such as reading the newspaper or watching television'}


<ipython-input-37-f7925241e3e7>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eighty_to_ninty_df = eighty_to_ninty_df.append({
Processing Rows: 100%|██████████| 90/90 [47:04<00:00, 21.12s/it]

The result is: True and {
  "reason": "The given label is 'Trouble falling or staying asleep or sleeping too much'. However, there is no mention or indication of any sleep-related issues in the post content. The post mainly focuses on the individual's struggle with depression and their efforts to find helpful habits and understand the concept of 'you' time. Therefore, it can be inferred that the given label is not applicable to the post content.",
  "value": "no",
  "post_title": "What is 'you' time?",
  "label": "Trouble falling or staying asleep or sleeping too much"
}
{'reason': "The given label is 'Trouble falling or staying asleep or sleeping too much'. However, there is no mention or indication of any sleep-related issues in the post content. The post mainly focuses on the individual's struggle with depression and their efforts to find helpful habits and understand the concept of 'you' time. Therefore, it can be inferred that the given label is not applicable to the post content.

In [ ]:
eighty_to_ninty_df.to_csv('eighty_to_nighty_data.csv', index=False)

## 90 to 100

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
ninty_to_hundred_df = pd.DataFrame(columns=columns)

In [ ]:
ninty_to_hundred_rows =  primate_input_df.iloc[90:100]

total_rows = len(ninty_to_hundred_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in ninty_to_hundred_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            ninty_to_hundred_df = ninty_to_hundred_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with half minute interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    ninty_to_hundred_df = ninty_to_hundred_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      progress_bar.update(1)

Processing Rows:   0%|          | 0/90 [00:00<?, ?it/s]

The result is: True and {
  "post_title": "What happens when you text a suicide Hotline?",
  "post_text": "Suicidal thoughts are a thing I deal with all the time, especially when I end up feeling low because I have no motivation for anything. My therapists recommend using a Hotline or whatever when it gets really really bad, but I just don't see the point?\n\nI'm not going to kill myself, it's too much effort. And it's just thoughts, so it feels kind of silly to text a crises line.\n\nSo I'm curious what actually happens when you use one?",
  "label": "Feeling bad about yourself or that you are a failure or have let yourself or your family down",
  "reason": "The post text mentions feeling low, having no motivation, and dealing with suicidal thoughts. These thoughts are related to feeling bad about oneself, which aligns with the given label."
}
{'post_title': 'What happens when you text a suicide Hotline?', 'post_text': "Suicidal thoughts are a thing I deal with all the time, especiall

<ipython-input-10-816130ca1d55>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ninty_to_hundred_df = ninty_to_hundred_df.append({
Processing Rows:   1%|          | 1/90 [00:42<1:03:26, 42.77s/it]

{'reason': 'The post text mentions feeling low, lack of motivation, and suicidal thoughts, which are indicative of feeling bad about oneself.', 'value': 'yes', 'post_title': 'What happens when you text a suicide Hotline?', 'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down'}
attempt 1 was SUCCESSFUL


<ipython-input-10-816130ca1d55>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ninty_to_hundred_df = ninty_to_hundred_df.append({
Processing Rows:   2%|▏         | 2/90 [00:47<30:04, 20.50s/it]  

...

{'reason': 'The post text indicates feelings of being a burden, wanting to die, and lacking social support.', 'value': 'yes', 'post_title': 'I feel alone, and a burden to everyone.', 'label': 'Thoughts that you would be better off dead or of hurting yourself in some way'}
attempt 2 was SUCCESSFUL
The result is: True and {
  "post_title": "I feel alone , and a burden to everyone.",
  "post_text": "I have had depression since I was 15 , started treatment at 18. Now I'm 21 and I know my mom's tired . She doesn't know what to do anymore and neither do I. I feel like I'm a burden. That I'm taking space of someone else. I feel like I should just fucking die and leave everyone alone. I want to talk to my ex because I want to feel the way he loved me again. But I know he hates me too, or doesn't even think about me anyway. I want someone to just hug me and let me cry in their arms. But I know I fucked things up and nobody can't stand me now. I don't fit in. I have no friends , no one I can't t

<ipython-input-10-816130ca1d55>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ninty_to_hundred_df = ninty_to_hundred_df.append({
Processing Rows:  99%|█████████▉| 89/90 [40:25<00:38, 38.78s/it]

{'reason': "The given label 'Trouble concentrating on things such as reading the newspaper or watching television' is not directly mentioned in the post content or title.", 'value': 'no', 'post_title': 'I feel alone , and a burden to everyone.', 'label': 'Trouble concentrating on things such as reading the newspaper or watching television'}
attempt 1 was SUCCESSFUL


<ipython-input-10-816130ca1d55>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ninty_to_hundred_df = ninty_to_hundred_df.append({
Processing Rows: 100%|██████████| 90/90 [40:30<00:00, 28.55s/it]

The result is: True and {
  "reason": "The post text indicates feelings of loneliness, burden, and thoughts of death, but doesn't specifically mention trouble falling or staying asleep. Therefore, based on the given post content, it is not clear whether the mentioned label/question would be applicable.",
  "value": "no",
  "post_title": "I feel alone , and a burden to everyone.",
  "label": "Trouble falling or staying asleep or sleeping too much"
}
{'reason': "The post text indicates feelings of loneliness, burden, and thoughts of death, but doesn't specifically mention trouble falling or staying asleep. Therefore, based on the given post content, it is not clear whether the mentioned label/question would be applicable.", 'value': 'no', 'post_title': 'I feel alone , and a burden to everyone.', 'label': 'Trouble falling or staying asleep or sleeping too much'}


In [ ]:
ninty_to_hundred_df.to_csv('nighty_to_hundred_data.csv', index=False)

## 100 to 110(New)

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
hundred_to_hundred_ten_df = pd.DataFrame(columns=columns)

In [ ]:
hundred_to_hundred_ten_rows =  primate_input_df.iloc[100:110]

total_rows = len(hundred_to_hundred_ten_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in hundred_to_hundred_ten_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            hundred_to_hundred_ten_df = hundred_to_hundred_ten_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 30 seconds interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    hundred_to_hundred_ten_df = hundred_to_hundred_ten_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      progress_bar.update(1)

Processing Rows:   0%|          | 0/90 [00:00<?, ?it/s]

The result is: True and {
  "post_title": "Please help, I'm so alone",
  "post_text": "I am so alone. Not just lonely, completely alone. There's not one person I can turn to for help. I don't know what I did to deserve this suffering. I don't understand what makes me so repulsive that not one single person wants to be in my life, despite trying my absolute hardest. I don't understand what I can do to have a normal life and nobody will help.\n\nEveryone says I just have to wait and be patient and \"people will come\", but they haven't, and life doesn't wait for anybody. All this time keeps ticking and going down the drain and I've missed out on so much. I am 25 now and still haven't even learned how to be a teenager or had any teenage friends or teenage life experiences yet, but now I'm too old to even have those experiences and have completely missed out on everything.\n\nI can't even get therapy because they said I have to have at least 1 hour a week of social interaction for 6 months

<ipython-input-10-cd9ccb37b251>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_to_hundred_ten_df = hundred_to_hundred_ten_df.append({
Processing Rows:   1%|          | 1/90 [01:07<1:39:57, 67.39s/it]

{'reason': 'The post text expresses feelings of worthlessness, loneliness, and a sense of failure. The individual feels they have let themselves down and lacks support from others.', 'value': 'yes', 'post_title': "Please help, I'm so alone", 'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down'}
attempt 1 was SUCCESSFUL


<ipython-input-10-cd9ccb37b251>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_to_hundred_ten_df = hundred_to_hundred_ten_df.append({
Processing Rows:   2%|▏         | 2/90 [01:12<45:31, 31.04s/it]  

...

The result is: True and {
  "reason": "The post text indicates that the person is experiencing thoughts of self-harm, which aligns with the given label.",
  "value": "yes",
  "post_title": "Tonight was rough",
  "label": "Thoughts that you would be better off dead or of hurting yourself in some way"
}
{'reason': 'The post text indicates that the person is experiencing thoughts of self-harm, which aligns with the given label.', 'value': 'yes', 'post_title': 'Tonight was rough', 'label': 'Thoughts that you would be better off dead or of hurting yourself in some way'}
The result is: True and {
  "post_title": "Tonight was rough",
  "post_text": "Idk if i should put a trigger warning on this, and tbh I'm not really sure how too. So uh....\n\n!TRIGGER WARNING! \n\nIf you struggles with eating disorders and suicidal ideation, uhm, this might be hard to read. Uhm. Idk if that's good or not. If not I'm sorry. Truth be told if i don't get this off my chest now im afraid of what I'll do\n\nI cri

<ipython-input-10-cd9ccb37b251>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_to_hundred_ten_df = hundred_to_hundred_ten_df.append({
Processing Rows:  99%|█████████▉| 89/90 [1:08:43<01:04, 64.48s/it]

{'reason': 'The post text does not mention trouble concentrating on things such as reading the newspaper or watching television. The post mainly focuses on struggles with eating disorders, suicidal ideation, depression, and self-abuse.', 'value': 'no', 'post_title': 'Tonight was rough', 'label': 'Trouble concentrating on things such as reading the newspaper or watching television'}
attempt 2 was SUCCESSFUL
The result is: True and {
  "post_title": "Tonight was rough",
  "post_text": "Idk if i should put a trigger warning on this, and tbh I'm not really sure how too. So uh....\n\n!TRIGGER WARNING!\n\nIf you struggles with eating disorders and suicidal ideation, uhm, this might be hard to read. Uhm. Idk if that's good or not. If not I'm sorry. Truth be told if i don't get this off my chest now im afraid of what I'll do\n\nI cried so hard i threw up tonight. TBH it felt good. In a scary way. Due to depression and mental and emotional abuse, a few years ago I was severely malnourished. My 

<ipython-input-10-cd9ccb37b251>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_to_hundred_ten_df = hundred_to_hundred_ten_df.append({
Processing Rows: 100%|██████████| 90/90 [1:10:06<00:00, 70.09s/it]

{'post_title': 'Tonight was rough', 'post_text': 'Idk if i should put a trigger warning on this, and tbh I\'m not really sure how too. So uh....\n\n!TRIGGER WARNING! \n\nIf you struggles with eating disorders and suicidal ideation, uhm, this might be hard to read. Uhm. Idk if that\'s good or not. If not I\'m sorry. Truth be told if i don\'t get this off my chest now im afraid of what I\'ll do\n\nI cried so hard i threw up tonight. TBH it felt good. In a scary way. Due to depression and mental and emotional abuse, a few years ago I was severely malnourished. My hair was falling out. I had growths on the back of my tongue. My nails flaked a part. I stopped eating almost completely. \n\nI\'m now clinically obese. Not excessively so, but jumping from 130 to 230 in 3 months was a fucking wild ride. I dont want to go back to that place. Im gonna talk to my therapist about it in a few days, but I couldn\'t hold it in anymore. \n\nI\'m scared. Im scared of what I\'ll do to feel better, to hate

In [ ]:
hundred_to_hundred_ten_df.to_csv('100_to_110_data.csv', index=False)

## 110 to 150(New)

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
hundred_ten_to_hundred_fifty_df = pd.DataFrame(columns=columns)

In [ ]:
hundred_to_hundred_fifty_rows =  primate_input_df.iloc[110:150]

total_rows = len(hundred_to_hundred_fifty_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in hundred_to_hundred_fifty_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            hundred_ten_to_hundred_fifty_df = hundred_ten_to_hundred_fifty_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 30 seconds interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    hundred_ten_to_hundred_fifty_df = hundred_ten_to_hundred_fifty_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      progress_bar.update(1)

Processing Rows:   0%|          | 0/360 [00:11<?, ?it/s]


The result is: True and {
  "post_title": "Had my 1st doctor appointment to address my anxiety/depression, it was......weird. Need some feedback, please.",
  "post_text": "So I had my first Dr's appointment to address my increasing anxiety and depression yesterday, and found some things a bit...odd. I'd like some feedback on a few things (just need to know whether some of the stuff I'm about to post is common, or not)",
  "label": "Feeling bad about yourself or that you are a failure or have let yourself or your family down",
  "reason": "The post text and post title do not directly discuss feeling bad about oneself or being a failure. The focus is on the weirdness of the doctor appointment and seeking feedback on certain aspects."
}
{'post_title': 'Had my 1st doctor appointment to address my anxiety/depression, it was......weird. Need some feedback, please.', 'post_text': "So I had my first Dr's appointment to address my increasing anxiety and depression yesterday, and found some thin

<ipython-input-14-7ef968e36206>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_ten_to_hundred_fifty_df = hundred_ten_to_hundred_fifty_df.append({
Processing Rows:   0%|          | 1/360 [03:30<20:58:27, 210.33s/it]

{'reason': 'The post text mentions the experience of feeling odd, not knowing how to answer questions, and not feeling confident about the prescribed medication. This aligns with the feeling of failure or letting oneself down.', 'value': 'Yes', 'post_title': 'Had my 1st doctor appointment to address my anxiety/depression, it was......weird. Need some feedback, please.', 'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down'}
attempt 3 was SUCCESSFUL
The result is: True and {
  "post_title": "Had my 1st doctor appointment to address my anxiety/depression, it was......weird. Need some feedback, please.",
  "post_text": "So I had my first Dr's appointment to address my increasing anxiety and depression yesterday, and found some things a bit...odd. I'd like some feedback on a few things (just need to know whether some of the stuff I'm about to post is common, or not)...\n\n1.) A lot of the stuff was \"up to me\" and I was given questions I 

<ipython-input-14-7ef968e36206>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_ten_to_hundred_fifty_df = hundred_ten_to_hundred_fifty_df.append({
Processing Rows:   1%|          | 2/360 [06:22<18:40:46, 187.84s/it]

...

The result is: True and {
  "post_title": "Seriously considering checking myself into a hospital...",
  "post_text": "Has anyone ever checked themselves in before? I've been trying to research what they do but it's really scary and hard for me to read about it (I'm having a really hard time focusing and also all the legalese is kinda confusing to understand). I just changed my meds from lexapro and wellbutrin to effexor and I'm sure that has something to with the bathroom cleaners looking like a tasty cocktail... I'm kinda scared to be by myself but even more afraid to be locked up and isolated in a hospital. not to mention hospital expenses.... i dont know what to do...",
  "label": "Thoughts that you would be better off dead or of hurting yourself in some way",
  "reason": "The post text mentions being scared, feeling isolated, and having thoughts about self-harm.",
  "value": "yes"
}
{'post_title': 'Seriously considering checking myself into a hospital...', 'post_text': "Has anyone 

<ipython-input-14-7ef968e36206>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_ten_to_hundred_fifty_df = hundred_ten_to_hundred_fifty_df.append({
Processing Rows: 100%|█████████▉| 359/360 [2:40:06<00:22, 22.08s/it]

The result is: True and {
  "post_title": "Seriously considering checking myself into a hospital...",
  "post_text": "Has anyone ever checked themselves in before? I've been trying to research what they do but it's really scary and hard for me to read about it (I'm having a really hard time focusing and also all the legalese is kinda confusing to understand). I just changed my meds from lexapro and wellbutrin to effexor and I'm sure that has something to with the bathroom cleaners looking like a tasty cocktail... I'm kinda scared to be by myself but even more afraid to be locked up and isolated in a hospital. not to mention hospital expenses.... i dont know what to do...",
  "label": "Trouble concentrating on things such as reading the newspaper or watching television",
  "reason": "The given post text indicates that the person is having a hard time focusing, finding it scary and hard to read, and mentioning trouble concentrating on things. Therefore, it is reasonable to infer that the

<ipython-input-14-7ef968e36206>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_ten_to_hundred_fifty_df = hundred_ten_to_hundred_fifty_df.append({
Processing Rows: 100%|██████████| 360/360 [2:40:10<00:00, 16.84s/it]

The result is: True and {
  "reason": "The given label/question is not directly related to the content of the post text or title.",
  "value": "no",
  "post_title": "Seriously considering checking myself into a hospital...",
  "label": "Trouble falling or staying asleep or sleeping too much"
}
{'reason': 'The given label/question is not directly related to the content of the post text or title.', 'value': 'no', 'post_title': 'Seriously considering checking myself into a hospital...', 'label': 'Trouble falling or staying asleep or sleeping too much'}


In [ ]:
hundred_ten_to_hundred_fifty_df.to_csv('110_to_150_data.csv', index=False)

## 150 to 180(New)

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
hundred_fifty_to_hundred_eighty_df = pd.DataFrame(columns=columns)

In [ ]:
hundred_fifty_to_hundred_eighty_rows =  primate_input_df.iloc[150:180]

total_rows = len(hundred_fifty_to_hundred_eighty_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in hundred_fifty_to_hundred_eighty_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            hundred_fifty_to_hundred_eighty_df = hundred_fifty_to_hundred_eighty_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 30 seconds interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    hundred_fifty_to_hundred_eighty_df = hundred_fifty_to_hundred_eighty_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      progress_bar.update(1)

Processing Rows:   0%|          | 0/270 [00:00<?, ?it/s]<ipython-input-12-978352bb5c39>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_fifty_to_hundred_eighty_df = hundred_fifty_to_hundred_eighty_df.append({
Processing Rows:   0%|          | 1/270 [00:04<21:34,  4.81s/it]

The result is: True and {
  "reason": "The post text mentions the lack of energy and struggling with daily tasks due to depression, which aligns with the label 'Feeling bad about yourself or that you are a failure or have let yourself or your family down'.",
  "value": "yes",
  "post_title": "Trying to have energy",
  "label": "Feeling bad about yourself or that you are a failure or have let yourself or your family down"
}
{'reason': "The post text mentions the lack of energy and struggling with daily tasks due to depression, which aligns with the label 'Feeling bad about yourself or that you are a failure or have let yourself or your family down'.", 'value': 'yes', 'post_title': 'Trying to have energy', 'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down'}


<ipython-input-12-978352bb5c39>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_fifty_to_hundred_eighty_df = hundred_fifty_to_hundred_eighty_df.append({
Processing Rows:   1%|          | 2/270 [00:09<21:26,  4.80s/it]

The result is: True and {
  "reason": "The post text mentions feeling down, lacking energy, and struggling with depression. The author also expresses a desire for tips on how to get more energy despite their depression. This suggests that the problem mentioned in the label, 'Feeling down depressed or hopeless,' does exist in the post content.",
  "value": "yes",
  "post_title": "Trying to have energy",
  "label": "Feeling down depressed or hopeless"
}
{'reason': "The post text mentions feeling down, lacking energy, and struggling with depression. The author also expresses a desire for tips on how to get more energy despite their depression. This suggests that the problem mentioned in the label, 'Feeling down depressed or hopeless,' does exist in the post content.", 'value': 'yes', 'post_title': 'Trying to have energy', 'label': 'Feeling down depressed or hopeless'}


<ipython-input-12-978352bb5c39>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_fifty_to_hundred_eighty_df = hundred_fifty_to_hundred_eighty_df.append({
Processing Rows:   1%|          | 3/270 [00:13<19:43,  4.43s/it]

...

{'reason': "The given label/question is not related to the post content. The post is about the user's uncertainty and feeling unqualified to talk about mental health content on YouTube, but it does not mention any thoughts of self-harm or wanting to die.", 'value': 'no', 'post_title': "I want to make a mental health channel on YouTube, but don't feel qualified...", 'label': 'Thoughts that you would be better off dead or of hurting yourself in some way'}
attempt 3 was SUCCESSFUL


<ipython-input-12-978352bb5c39>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_fifty_to_hundred_eighty_df = hundred_fifty_to_hundred_eighty_df.append({
Processing Rows: 100%|█████████▉| 269/270 [1:58:06<00:52, 52.36s/it]

The result is: True and {
  "post_title": "I want to make a mental health channel on YouTube, but don't feel qualified...",
  "post_text": "Let me start by saying, I'm not in it for the views. Just that, recently I did a vlog after jotting down some things that help me when I'm depressed. It felt sort of cathartic to write down things that help me, and hope it could help other people. My problem is I feel like what I'll contribute will be generic, or not very helpful since everyone deals with depression differently.\n\nI'm not making this post to advertise my channel. (throwaway acc) I'm just unsure I should post because I don't feel quite qualified to talk about the content. Sorry if this isn't entirely relevant to the subreddit.",
  "label": "Trouble concentrating on things such as reading the newspaper or watching television",
  "value": "no",
  "reason": "The post text and title do not mention any difficulty in concentrating on reading the newspaper or watching television."
}
{'pos

<ipython-input-12-978352bb5c39>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hundred_fifty_to_hundred_eighty_df = hundred_fifty_to_hundred_eighty_df.append({
Processing Rows: 100%|██████████| 270/270 [1:58:11<00:00, 38.26s/it]

The result is: True and {
  "reason": "The given label does not directly mention any trouble related to falling asleep or sleeping too much. The post text focuses on the user's hesitation and insecurity about starting a mental health channel on YouTube. There is no explicit mention of sleep-related problems in the content.",
  "value": "no",
  "post_title": "I want to make a mental health channel on YouTube, but don't feel qualified...",
  "label": "Trouble falling or staying asleep or sleeping too much"
}
{'reason': "The given label does not directly mention any trouble related to falling asleep or sleeping too much. The post text focuses on the user's hesitation and insecurity about starting a mental health channel on YouTube. There is no explicit mention of sleep-related problems in the content.", 'value': 'no', 'post_title': "I want to make a mental health channel on YouTube, but don't feel qualified...", 'label': 'Trouble falling or staying asleep or sleeping too much'}


In [ ]:
hundred_fifty_to_hundred_eighty_df.to_csv('150_to_180_data.csv', index=False)

## 180 to 200(New)

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
hundred_eighty_to_two_hundred_df = pd.DataFrame(columns=columns)

In [ ]:
hundred_eighty_to_two_hundred_rows =  primate_input_df.iloc[180:200]

total_rows = len(hundred_eighty_to_two_hundred_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in hundred_eighty_to_two_hundred_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            hundred_eighty_to_two_hundred_df = hundred_eighty_to_two_hundred_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 30 seconds interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    hundred_eighty_to_two_hundred_df = hundred_eighty_to_two_hundred_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      progress_bar.update(1)

## 200 to 225(New)

## 225 to 250(New)

In [8]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
_225_to_250_df = pd.DataFrame(columns=columns)

In [10]:
_225_to_250_rows =  primate_input_df.iloc[225:250]

total_rows = len(_225_to_250_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in _225_to_250_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        try:
            load_gpt_ans_to_json = json.loads(gpt_answer)
            print(load_gpt_ans_to_json)
            _225_to_250_df = _225_to_250_df.append({
              'post_title': post_title,
              'label': value,
              'value': load_gpt_ans_to_json['value'],
              'reason': load_gpt_ans_to_json['reason']
            }, ignore_index=True)
            result = {
              "bool": boolean,
              "json": load_gpt_ans_to_json
            }

        except Exception as e:
            print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 30 seconds interval...')
            # attempt 1 to 3 more times with 0.5 minute interval to succeed
            for i in range(3):
                try:
                    time.sleep(30)
                    print('attempt ' + str(i+1) + ' out of 3')
                    boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, value)
                    load_gpt_ans_to_json = json.loads(gpt_answer)
                    print(load_gpt_ans_to_json)
                    _225_to_250_df = _225_to_250_df.append({
                      'post_title': post_title,
                      'label': value,
                      'value': load_gpt_ans_to_json['value'],
                      'reason': load_gpt_ans_to_json['reason']
                    }, ignore_index=True)
                    result = {
                      "bool": boolean,
                      "json": load_gpt_ans_to_json
                    }
                    print('attempt ' + str(i+1) + ' was SUCCESSFUL')
                    # break if succeed
                    break
                except Exception as e:
                    print('attempt ' + str(i + 1) + ' FAILED')
                    print(e)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      progress_bar.update(1)


Processing Rows:   0%|          | 0/225 [01:32<?, ?it/s]
<ipython-input-10-7dbd31ce4e13>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _225_to_250_df = _225_to_250_df.append({

Processing Rows:   0%|          | 1/225 [00:11<44:21, 11.88s/it]

The result is: True and {
  "post_title": "Anyone here have reverse seasonal affective disorder - i.e.: summer depression? If so, could use some advice",
  "post_text": "For the past two summers like clock work around the last week in July I have been hit with the above - I don’t know what else it is other than reverse SAD. It usually starts with a general few months of feeling good and stable - meaning I’m feeling normal, motivated and generally happy with some dips that I’m used to for the most part. The past two summers I will have a really good day/week/ whatever and then one morning I’ll wake up and feel “hungover” or “in a fog” even though I haven’t done anything to bring this on (like drinking, using substances). Also, I’m quite active physically - so that’s also strange... [truncated for brevity]",
  "label": "Feeling bad about yourself or that you are a failure or have let yourself or your family down",
  "reason": "The given label does not directly relate to the content of th

<ipython-input-10-7dbd31ce4e13>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _225_to_250_df = _225_to_250_df.append({

Processing Rows:   1%|          | 2/225 [00:15<26:25,  7.11s/it]

The result is: True and {
  "reason": "The post text indicates the presence of seasonal affective disorder (SAD) symptoms during the summer, causing the individual to feel down and depressed.",
  "value": "yes",
  "post_title": "Anyone here have reverse seasonal affective disorder - i.e.: summer depression? If so, could use some advice",
  "label": "Feeling down depressed or hopeless"
}
{'reason': 'The post text indicates the presence of seasonal affective disorder (SAD) symptoms during the summer, causing the individual to feel down and depressed.', 'value': 'yes', 'post_title': 'Anyone here have reverse seasonal affective disorder - i.e.: summer depression? If so, could use some advice', 'label': 'Feeling down depressed or hopeless'}


<ipython-input-10-7dbd31ce4e13>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _225_to_250_df = _225_to_250_df.append({

Processing Rows:   1%|▏         | 3/225 [00:19<20:57,  5.67s/it]

...

The result is: True and {
  "reason": "The given label does not directly correlate with the content of the post. The post discusses struggles with body image and feelings of self-hatred, but it does not mention thoughts of self-harm or wanting to be dead.",
  "value": "no",
  "post_title": "I hate my body and my thoughts",
  "label": "Thoughts that you would be better off dead or of hurting yourself in some way"
}
{'reason': 'The given label does not directly correlate with the content of the post. The post discusses struggles with body image and feelings of self-hatred, but it does not mention thoughts of self-harm or wanting to be dead.', 'value': 'no', 'post_title': 'I hate my body and my thoughts', 'label': 'Thoughts that you would be better off dead or of hurting yourself in some way'}
The result is: True and {
  "post_title": "I hate my body and my thoughts",
  "post_text": "Ever since I was little I was always struggling with my physical appearance. I have been told throughout m

<ipython-input-10-7dbd31ce4e13>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _225_to_250_df = _225_to_250_df.append({

Processing Rows: 100%|█████████▉| 224/225 [1:14:34<00:21, 21.44s/it]

{'reason': "The given label is not related to the post content. The post discusses the author's struggles with their physical appearance and self-perception, but does not mention any difficulty concentrating on reading the newspaper or watching television.", 'value': 'no', 'post_title': 'I hate my body and my thoughts', 'label': 'Trouble concentrating on things such as reading the newspaper or watching television'}
attempt 1 was SUCCESSFUL
The result is: True and {
  "post_title": "I hate my body and my thoughts",
  "post_text": "Ever since I was little I was always struggling with my physical appearance. I have been told throughout my life that I was overweight and that just screwed over my image even more. Here I am feeling guilty cause I had a bit of grill chicken and honestly I'm just tired. I'm tired of thinking of exercise as a mere chore. I'm tired that I can't enjoy food anymore, and the fact that my mom cooks so much doesn't help. My family all says I'm lean and normal but I f

<ipython-input-10-7dbd31ce4e13>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _225_to_250_df = _225_to_250_df.append({

Processing Rows: 100%|██████████| 225/225 [1:15:20<00:00, 28.69s/it]

{'reason': "The given label does not relate to the problem described in the post content. The post is about the person's struggle with body image and self-esteem, not about trouble falling or staying asleep or sleeping too much.", 'value': 'no', 'post_title': 'I hate my body and my thoughts', 'label': 'Trouble falling or staying asleep or sleeping too much'}
attempt 1 was SUCCESSFUL


In [11]:
_225_to_250_df.to_csv('225_to_250_data.csv', index=False)

# Pivoting and Reverse the main dataframe

In [8]:
primate_input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2003 entries, 0 to 2002
Data columns (total 11 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   post_title                                                                                                                                                             2003 non-null   object 
 1   post_text                                                                                                                                                              2003 non-null   object 
 2   Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down           

In [9]:
labels = list(primate_input_df.columns[2:])

primate_data_single_label = pd.DataFrame(columns=["post_title", "post_text", "label", "value"])

for index, row in primate_input_df.iterrows():
  for label in labels:
    new_row = {"post_title": row["post_title"],
               "post_text": row["post_text"],
               "label": label,
               "value": row[label]}
    # new_df = new_df.append(new_row, ignore_index=True)
    primate_data_single_label = pd.concat([primate_data_single_label,
                                           pd.DataFrame(new_row, index=[0])],
                                           ignore_index=True)


primate_data_single_label.to_csv("primate_data_single_label.csv", index=False)

In [10]:
primate_data_single_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18027 entries, 0 to 18026
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  18027 non-null  object
 1   post_text   18027 non-null  object
 2   label       18027 non-null  object
 3   value       18021 non-null  object
dtypes: object(4)
memory usage: 563.5+ KB


In [ ]:
primate_data_single_label.head()

,post_title,post_text,label,value
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-bad-about-yourself-or-that-you-are-a-f...,1
1,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-down-depressed-or-hopeless,0.0
2,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-tired-or-having-little-energy,1
3,I don't feel original anymore.,"When I was in high school a few years back, I ...",Little-interest-or-pleasure-in-doing,1
4,I don't feel original anymore.,"When I was in high school a few years back, I ...",Moving-or-speaking-so-slowly-that-other-people...,0


In [11]:
primate_data_single_label['label'] = primate_data_single_label['label'].str.replace('-', ' ')

In [ ]:
primate_data_single_label.head(10)

,post_title,post_text,label,value
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling bad about yourself or that you are a f...,1
1,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling down depressed or hopeless,0.0
2,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling tired or having little energy,1
3,I don't feel original anymore.,"When I was in high school a few years back, I ...",Little interest or pleasure in doing,1
4,I don't feel original anymore.,"When I was in high school a few years back, I ...",Moving or speaking so slowly that other people...,0
5,I don't feel original anymore.,"When I was in high school a few years back, I ...",Poor appetite or overeating,0.0
6,I don't feel original anymore.,"When I was in high school a few years back, I ...",Thoughts that you would be better off dead or ...,0.0
7,I don't feel original anymore.,"When I was in high school a few years back, I ...",Trouble concentrating on things such as readin...,0.0
8,I don't feel original anymore.,"When I was in high school a few years back, I ...",Trouble falling or staying asleep or sleeping ...,0
9,I don’t have anyone to talk to and I don’t kno...,Nine years ago I was diagnosed with depression...,Feeling bad about yourself or that you are a f...,1


In [12]:
primate_data_single_label['label'] = primate_data_single_label['label'].str.strip()

In [ ]:
primate_data_single_label.head(10)

,post_title,post_text,label,value
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling bad about yourself or that you are a f...,1
1,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling down depressed or hopeless,0.0
2,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling tired or having little energy,1
3,I don't feel original anymore.,"When I was in high school a few years back, I ...",Little interest or pleasure in doing,1
4,I don't feel original anymore.,"When I was in high school a few years back, I ...",Moving or speaking so slowly that other people...,0
5,I don't feel original anymore.,"When I was in high school a few years back, I ...",Poor appetite or overeating,0.0
6,I don't feel original anymore.,"When I was in high school a few years back, I ...",Thoughts that you would be better off dead or ...,0.0
7,I don't feel original anymore.,"When I was in high school a few years back, I ...",Trouble concentrating on things such as readin...,0.0
8,I don't feel original anymore.,"When I was in high school a few years back, I ...",Trouble falling or staying asleep or sleeping ...,0
9,I don’t have anyone to talk to and I don’t kno...,Nine years ago I was diagnosed with depression...,Feeling bad about yourself or that you are a f...,1


## true 50 to 60

In [ ]:
true_sixth_90_df = primate_data_single_label[450:540]
true_sixth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 450 to 539
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   post_text   90 non-null     object
 2   label       90 non-null     object
 3   value       90 non-null     object
dtypes: object(4)
memory usage: 2.9+ KB


In [ ]:
true_sixth_90_df.head(n=20)

,post_title,post_text,label,value
450,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Feeling bad about yourself or that you are a f...,1
451,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Feeling down depressed or hopeless,1.0
452,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Feeling tired or having little energy,1
453,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Little interest or pleasure in doing,1
454,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Moving or speaking so slowly that other people...,0
455,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Poor appetite or overeating,1.0
456,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Thoughts that you would be better off dead or ...,1.0
457,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Trouble concentrating on things such as readin...,0.0
458,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Trouble falling or staying asleep or sleeping ...,0
459,Lost in a rabbit hole @_@,"Hi, I'm 23 years old and I tend to feel like I...",Feeling bad about yourself or that you are a f...,1


## true 60 to 80

In [ ]:
true_sixth_seventh_90_df = primate_data_single_label[540:720]
true_sixth_seventh_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 540 to 719
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  180 non-null    object
 1   post_text   180 non-null    object
 2   label       180 non-null    object
 3   value       180 non-null    object
dtypes: object(4)
memory usage: 5.8+ KB


In [ ]:
true_sixth_seventh_90_df.head(n=20)

,post_title,post_text,label,value
540,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Feeling bad about yourself or that you are a f...,1
541,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Feeling down depressed or hopeless,1.0
542,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Feeling tired or having little energy,0
543,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Little interest or pleasure in doing,0
544,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Moving or speaking so slowly that other people...,0
545,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Poor appetite or overeating,0.0
546,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Thoughts that you would be better off dead or ...,0.0
547,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Trouble concentrating on things such as readin...,0.0
548,What activities do you like to do to face your...,Hi all. I've been dealing with my D&amp;A for ...,Trouble falling or staying asleep or sleeping ...,0
549,Cutting,"Hello,\n\nYesterday I cut myself, on my arm. I...",Feeling bad about yourself or that you are a f...,1


## true 80 to 90

In [ ]:
true_eighth_90_df = primate_data_single_label[720:810]
true_eighth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 720 to 809
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   post_text   90 non-null     object
 2   label       90 non-null     object
 3   value       90 non-null     object
dtypes: object(4)
memory usage: 2.9+ KB


In [ ]:
true_eighth_90_df.head(20)

,post_title,post_text,label,value
720,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Feeling bad about yourself or that you are a f...,1
721,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Feeling down depressed or hopeless,0.0
722,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Feeling tired or having little energy,1
723,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Little interest or pleasure in doing,1
724,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Moving or speaking so slowly that other people...,0
725,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Poor appetite or overeating,0.0
726,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Thoughts that you would be better off dead or ...,0.0
727,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Trouble concentrating on things such as readin...,0.0
728,I am a toxic person and I want to not be like ...,I am a 27 year old man who has recently come t...,Trouble falling or staying asleep or sleeping ...,0
729,I'm stuck here and nobody likes me (22 F),(I am a 22 year old English teacher living in ...,Feeling bad about yourself or that you are a f...,1


## true 90 to 100

In [ ]:
true_ninth_90_df = primate_data_single_label[810:900]
true_ninth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 810 to 899
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   post_text   90 non-null     object
 2   label       90 non-null     object
 3   value       90 non-null     object
dtypes: object(4)
memory usage: 2.9+ KB


In [ ]:
true_ninth_90_df.head(20)

,post_title,post_text,label,value
810,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Feeling bad about yourself or that you are a f...,1
811,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Feeling down depressed or hopeless,0.0
812,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Feeling tired or having little energy,0
813,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Little interest or pleasure in doing,0
814,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Moving or speaking so slowly that other people...,0
815,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Poor appetite or overeating,0.0
816,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Thoughts that you would be better off dead or ...,1.0
817,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Trouble concentrating on things such as readin...,0.0
818,What happens when you text a suicide Hotline?,Suicidal thoughts are a thing I deal with all ...,Trouble falling or staying asleep or sleeping ...,0
819,GF and Depression,I’m going through some rough stuff with my dep...,Feeling bad about yourself or that you are a f...,1


## true 100 to 150

In [15]:
true_100_to_150_df = primate_data_single_label[900:1350]
true_100_to_150_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 900 to 1349
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  450 non-null    object
 1   post_text   450 non-null    object
 2   label       450 non-null    object
 3   value       450 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


In [18]:
true_100_to_150_df.head()

,post_title,post_text,label,value
900,"Please help, I'm so alone","I am so alone. Not just lonely, completely alo...",Feeling bad about yourself or that you are a f...,1
901,"Please help, I'm so alone","I am so alone. Not just lonely, completely alo...",Feeling down depressed or hopeless,1.0
902,"Please help, I'm so alone","I am so alone. Not just lonely, completely alo...",Feeling tired or having little energy,0
903,"Please help, I'm so alone","I am so alone. Not just lonely, completely alo...",Little interest or pleasure in doing,1
904,"Please help, I'm so alone","I am so alone. Not just lonely, completely alo...",Moving or speaking so slowly that other people...,0


## true 150 to 200

In [16]:
true_150_to_200_df = primate_data_single_label[1350:1800]
true_150_to_200_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 1350 to 1799
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  450 non-null    object
 1   post_text   450 non-null    object
 2   label       450 non-null    object
 3   value       450 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


## true 200 to 250

In [17]:
true_200_to_250_df = primate_data_single_label[1800:2250]
true_200_to_250_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 1800 to 2249
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  450 non-null    object
 1   post_text   450 non-null    object
 2   label       450 non-null    object
 3   value       450 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


# Merging True DFs

In [55]:
concatenated_true__df = pd.concat([true_sixth_90_df,
                                   true_sixth_seventh_90_df,
                                   true_eighth_90_df,
                                   true_ninth_90_df])

NameError: ignored

In [ ]:
concatenated_true__df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 450 to 899
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  450 non-null    object
 1   post_text   450 non-null    object
 2   label       450 non-null    object
 3   value       450 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


# Integration and Some Checks

In [ ]:
path = '/content/drive/MyDrive/gpt-second-50-datasets'

## 50 to 60

In [ ]:
pred_sixth_90_df = pd.read_csv(f"{path}/sixth_ten_data.csv",  encoding='latin-1')
pred_sixth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  96 non-null     object
 1   label       96 non-null     object
 2   value       96 non-null     object
 3   reason      96 non-null     object
dtypes: object(4)
memory usage: 3.1+ KB


In [ ]:
distinct_values = pred_sixth_90_df['value'].unique()
distinct_values

array(['yes', 'no', 'Yes'], dtype=object)

In [ ]:
pred_sixth_90_df['value'] = pred_sixth_90_df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1})

In [ ]:
distinct_values = pred_sixth_90_df['value'].unique()
distinct_values

array([1, 0])

In [ ]:
pred_sixth_90_df.head(10)

,post_title,label,value,reason
0,I donât know if Iâm depressed.,Feeling bad about yourself or that you are a f...,1,The given label mentions feeling bad about one...
1,I donât know if Iâm depressed.,Feeling down depressed or hopeless,1,The post text mentions feeling terrible lately...
2,I donât know if Iâm depressed.,Feeling tired or having little energy,1,"The post text mentions a loss of appetite, lac..."
3,I donât know if Iâm depressed.,Little interest or pleasure in doing,1,"The post text mentions a loss of appetite, fee..."
4,I donât know if Iâm depressed.,Moving or speaking so slowly that other people...,0,The given label mentions a symptom related to ...
5,I donât know if Iâm depressed.,Poor appetite or overeating,1,The post text mentions a loss of appetite for ...
6,I donât know if Iâm depressed.,Thoughts that you would be better off dead or ...,1,The post text includes the statement 'I donâ...
7,I donât know if Iâm depressed.,Trouble concentrating on things such as readin...,0,The given label is about trouble concentrating...
8,I donât know if Iâm depressed.,Trouble falling or staying asleep or sleeping ...,0,The post text mentions a loss of appetite and ...
9,Lost in a rabbit hole @_@,Feeling bad about yourself or that you are a f...,1,The post text mentions feeling like wasting ti...


In [ ]:
pred_sixth_90_df['post_title'].unique()
# pred_sixth_90_df['post_title'].nunique()

array(['I donâ\x80\x99t know if Iâ\x80\x99m depressed.',
       'Lost in a rabbit hole @_@', 'I want to go home',
       'Should I use the psychological help service that my university provides for free?',
       'How do I muster the energy to cook and eat, especially at the end of the day?',
       '17 year old forced to stop Paxil 20mg cold turkey. Looking for Advice!',
       'I have been severely depressed and it is affecting my school work and looking for advice',
       'I donâ\x80\x99t know what to do', "I can't open up to anyone",
       "My friend is dying inside and I don't know how to help him"],
      dtype=object)

In [ ]:
true_sixth_90_df['post_title'].unique()

array(['I don’t know if I’m depressed.', 'Lost in a rabbit hole @_@',
       'I want to go home',
       'Should I use the psychological help service that my university provides for free?',
       'How do I muster the energy to cook and eat, especially at the end of the day?',
       '17 year old forced to stop Paxil 20mg cold turkey. Looking for Advice!',
       'I have been severely depressed and it is affecting my school work and looking for advice',
       'I don’t know what to do', "I can't open up to anyone",
       "My friend is dying inside and I don't know how to help him"],
      dtype=object)

## 60 to 80

In [ ]:
pred_seventh_eighth_90_df = pd.read_csv(f"{path}/sixty_to_eighty_data.csv",  encoding='latin-1')
pred_seventh_eighth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  179 non-null    object
 1   label       179 non-null    object
 2   value       179 non-null    object
 3   reason      179 non-null    object
dtypes: object(4)
memory usage: 5.7+ KB


In [ ]:
distinct_values = pred_seventh_eighth_90_df['value'].unique()
distinct_values

array(['no', 'yes', 'Yes'], dtype=object)

In [ ]:
pred_seventh_eighth_90_df['value'] = pred_seventh_eighth_90_df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1})

In [ ]:
pred_seventh_eighth_90_df.head(10)

,post_title,label,value,reason
0,What activities do you like to do to face your...,Feeling bad about yourself or that you are a f...,0,The post text describes a feeling of apathy an...
1,What activities do you like to do to face your...,Feeling down depressed or hopeless,1,"The post text mentions feeling apathetic, lack..."
2,What activities do you like to do to face your...,Feeling tired or having little energy,1,The post text mentions feeling apathetic and l...
3,What activities do you like to do to face your...,Little interest or pleasure in doing,1,The post text mentions feeling apathetic and l...
4,What activities do you like to do to face your...,Moving or speaking so slowly that other people...,0,"The given label refers to physical behaviors, ..."
5,What activities do you like to do to face your...,Poor appetite or overeating,0,The post text mentions apathy and lack of moti...
6,What activities do you like to do to face your...,Thoughts that you would be better off dead or ...,0,The given label discusses thoughts of self-har...
7,What activities do you like to do to face your...,Trouble concentrating on things such as readin...,0,The given label/question does not directly per...
8,What activities do you like to do to face your...,Trouble falling or staying asleep or sleeping ...,0,The given label is unrelated to the content of...
9,Cutting,Feeling bad about yourself or that you are a f...,1,The post text mentions that the person feels t...


In [ ]:
pred_seventh_eighth_90_df['post_title'].unique()

array(['What activities do you like to do to face your apathy?',
       'Cutting', 'I AM DONE', 'I feel myself slipping again',
       "I'm alone.",
       'I am lost.. without purpose.. I just need help, talk, anything really...',
       'Should I?',
       "Someone is telling me they are going to kill their self and I don't know what to do.",
       'My problems are ruining my relationships. (Iâ\x80\x99ve also realized that I really donâ\x80\x99t like myself.)',
       'Talk about depression in job interview?',
       'falling again please help', 'Teenager with no hope.',
       'I just want to stop existing', 'Why do I even bother?',
       'depressed piece of shit',
       "What do I tell my boyfriend when I'm in a depressive episode and he asks what's wrong?",
       "I am too sensitive and don't know what to do about it",
       'About ready to ghost my mom?', 'Should I even go to school?',
       'Spiraled into depression what do I do :('], dtype=object)

In [ ]:
pred_seventh_eighth_90_df['post_title'].nunique()

20

In [ ]:
true_sixth_seventh_90_df['post_title'].unique()

array(['What activities do you like to do to face your apathy?',
       'Cutting', 'I AM DONE', 'I feel myself slipping again',
       "I'm alone.",
       'I am lost.. without purpose.. I just need help, talk, anything really...',
       'Should I?',
       "Someone is telling me they are going to kill their self and I don't know what to do.",
       'My problems are ruining my relationships. (I’ve also realized that I really don’t like myself.)',
       'Talk about depression in job interview?',
       'falling again please help', 'Teenager with no hope.',
       'I just want to stop existing', 'Why do I even bother?',
       'depressed piece of shit',
       "What do I tell my boyfriend when I'm in a depressive episode and he asks what's wrong?",
       "I am too sensitive and don't know what to do about it",
       'About ready to ghost my mom?', 'Should I even go to school?',
       'Spiraled into depression what do I do :('], dtype=object)

In [ ]:
label_counts = true_sixth_seventh_90_df.groupby('post_title')['label'].nunique()

# Identify titles that do not have all nine distinct labels
missing_labels = {}
for title, count in label_counts.items():
    if count < len(pre_processed_labels):
        missing = set(pre_processed_labels.keys()) \
        - set(true_sixth_seventh_90_df[true_sixth_seventh_90_df['post_title'] == title]['label'])
        missing_labels[title] = missing

if not missing_labels:
    print("All titles have all nine distinct labels.")
else:
    print("Titles missing labels:")
    for title, missing in missing_labels.items():
        print(f"{title}: {missing}")

All titles have all nine distinct labels.


### checking for missed label

In [19]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
missed_data_df = pd.DataFrame(columns=columns)

In [53]:
post_text = "I was thinking that whem she returns everything will be normal. It got worse. I'm in a deeper rabbit hole than before. Things I wanted to say to her, my feeling of emptiness, melancholy, and being down moat of the time ia replaced by verbal nods, spacing out, and being blunt.\n\nI'm afraid that her 1 monrh vacation made ua distant and that bothers me. Maybe I'm overthinking things. She wanted some alone me time, but constantly messages and video chatting me. That's what I'm bothered about.  Hot and cold. It makes me think what I have done wrong.\n\nAl of this I keep to my self. This burdens me. This added to my self pity., questioning my worth, and me wanting to just end it.\n\nI wanted to sleep and die in my sleep so that my thoughts won't torment me anymore"
post_title = "She's back and I'm sinking again."

label = pre_processed_labels["Q4"]
boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, label)
# print(f"The result is: {boolean} and {gpt_answer}")
if boolean:
  try:
      load_gpt_ans_to_json = json.loads(gpt_answer)
      print(load_gpt_ans_to_json)
      missed_data_df = missed_data_df.append({
        'post_title': post_title,
        'label': label,
        'value': load_gpt_ans_to_json['value'],
        'reason': load_gpt_ans_to_json['reason']
      }, ignore_index=True)
      result = {
        "bool": boolean,
        "json": load_gpt_ans_to_json
      }

  except Exception as e:
      print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 7 minutes interval...')
      # attempt 1 to 3 more times with 0.5 minute interval to succeed
      for i in range(3):
          try:
              time.sleep(30)
              print('attempt ' + str(i+1) + ' out of 3')
              boolean, gpt_answer = get_answer_2023_11_28(post_title, post_text, label)
              load_gpt_ans_to_json = json.loads(gpt_answer)
              print(load_gpt_ans_to_json)
              missed_data_df = missed_data_df.append({
                'post_title': post_title,
                'label': label,
                'value': load_gpt_ans_to_json['value'],
                'reason': load_gpt_ans_to_json['reason']
              }, ignore_index=True)
              result = {
                "bool": boolean,
                "json": load_gpt_ans_to_json
              }
              print('attempt ' + str(i+1) + ' was SUCCESSFUL')
              # break if succeed
              break
          except Exception as e:
              print('attempt ' + str(i + 1) + ' FAILED')
              print(e)

if not boolean:
  result = {
      "bool": boolean,
      "error": gpt_answer
  }

{'post_title': "She's back and I'm sinking again.", 'post_text': "I was thinking that when she returns everything will be normal. It got worse. I'm in a deeper rabbit hole than before. Things I wanted to say to her, my feeling of emptiness, melancholy, and being down most of the time is replaced by verbal nods, spacing out, and being blunt.\n\nI'm afraid that her 1 month vacation made us distant and that bothers me. Maybe I'm overthinking things. She wanted some alone me time, but constantly messages and video chatting me. That's what I'm bothered about. Hot and cold. It makes me think what I have done wrong.\n\nAll of this I keep to myself. This burdens me. This added to my self-pity, questioning my worth, and me wanting to just end it.\n\nI wanted to sleep and die in my sleep so that my thoughts won't torment me anymore", 'label': 'Little interest or pleasure in doing'}
'value'
First attempt for getting answer from GPT faild, try 1 to 3 more times with 7 minutes interval...
attempt 1

<ipython-input-53-8d0190ebf5a5>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_data_df = missed_data_df.append({


In [54]:
missed_data_df.head(20)

,post_title,label,value,reason
0,Help with budding relationship,Feeling bad about yourself or that you are a f...,no,The given label/question is not directly relat...
1,Seeing the doc tomorrow about starting medicat...,Trouble falling or staying asleep or sleeping ...,no,The post text does not mention any issues rela...
2,Who should I talk to,Trouble concentrating on things such as readin...,no,The given label/question is related to trouble...
3,Had my 1st doctor appointment to address my an...,Trouble falling or staying asleep or sleeping ...,no,The given label(question) does not relate to t...
4,I need help.,Trouble falling or staying asleep or sleeping ...,yes,The post text indicates that the person is exp...
5,Meds help - Sertraline,Trouble concentrating on things such as readin...,Yes,"Based on the post text, the person is describi..."
6,nothing left,Trouble concentrating on things such as readin...,no,The given post text and post title mention fee...
7,Will i ever be enough for someone?,Trouble concentrating on things such as readin...,no,"Based on the given post text and title, there ..."
8,How do I stop fucking myself over?,Moving or speaking so slowly that other people...,no,The post text does not mention anything relate...
9,"Skipped school to kill myself, didn't go throu...",Feeling bad about yourself or that you are a f...,yes,The post text suggests that the person is feel...


## 80 to 90

In [ ]:
pred_eighth_90_df = pd.read_csv(f"{path}/eighty_to_nighty_data.csv",  encoding='latin-1')
pred_eighth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  88 non-null     object
 1   label       88 non-null     object
 2   value       88 non-null     object
 3   reason      88 non-null     object
dtypes: object(4)
memory usage: 2.9+ KB


In [ ]:
pred_eighth_90_df['post_title'].unique()

array(['I am a toxic person and I want to not be like this anymore.',
       "I'm stuck here and nobody likes me (22 F)",
       'Burned out and trying to come back from it',
       'My Friend Is Suffering...',
       "I feel like a failure, because I'm failing to get certain things done",
       "I'm seriously considering suicide again.",
       'How can I (32F) stop feeling like such a burden to my husband (36M)?',
       'I cant get help',
       "Gotten to a really weird place where I don't even want tv on in the background",
       'What is "you" time?'], dtype=object)

In [ ]:
true_eighth_90_df['post_title'].unique()

array(['I am a toxic person and I want to not be like this anymore.',
       "I'm stuck here and nobody likes me (22 F)",
       'Burned out and trying to come back from it',
       'My Friend Is Suffering...',
       "I feel like a failure, because I'm failing to get certain things done",
       "I'm seriously considering suicide again.",
       'How can I (32F) stop feeling like such a burden to my husband (36M)?',
       'I cant get help',
       "Gotten to a really weird place where I don't even want tv on in the background",
       'What is "you" time?'], dtype=object)

## 90 to 100

In [ ]:
pred_ninth_90_df = pd.read_csv(f"{path}/nighty_to_hundred_data.csv",  encoding='latin-1')
pred_ninth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  86 non-null     object
 1   label       86 non-null     object
 2   value       86 non-null     object
 3   reason      86 non-null     object
dtypes: object(4)
memory usage: 2.8+ KB


# Mounting preprocessed files

In [ ]:
pred_sixth_90_df_preprocessed = pd.read_csv(f"{path}/sixth_ten_data_preprocessed.csv",  encoding='latin-1')
pred_sixth_90_df_preprocessed.info()

pred_seventh_eighth_90_df_preprocessed = pd.read_csv(f"{path}/sixty_to_eighty_data_preprocessed.csv",  encoding='latin-1')
pred_seventh_eighth_90_df_preprocessed.info()


pred_eighth_90_df_preprocessed = pd.read_csv(f"{path}/eighty_to_nighty_data_preprocessed.csv",  encoding='latin-1')
pred_eighth_90_df_preprocessed.info()


pred_ninth_90_df_preprocessed = pd.read_csv(f"{path}/nighty_to_hundred_data_preprocessed.csv",  encoding='latin-1')
pred_ninth_90_df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label       90 non-null     object
 2   value       90 non-null     object
 3   reason      90 non-null     object
dtypes: object(4)
memory usage: 2.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  180 non-null    object
 1   label       180 non-null    object
 2   value       180 non-null    object
 3   reason      180 non-null    object
dtypes: object(4)
memory usage: 5.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label  

In [ ]:
distinct_values = pred_ninth_90_df_preprocessed['value'].unique()
distinct_values

array([1, 0])

In [ ]:
pred_ninth_90_df_preprocessed['value'] = pred_ninth_90_df_preprocessed['value'].replace({'yes': 1, 'no': 0, 'No': 0, 'Yes': 1})

# Merging Pred DFs

In [ ]:
concatenated_pred__df = pd.concat([pred_sixth_90_df_preprocessed,
                                   pred_seventh_eighth_90_df_preprocessed,
                                   pred_eighth_90_df_preprocessed,
                                   pred_ninth_90_df_preprocessed])

In [ ]:
concatenated_pred__df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 89
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  450 non-null    object
 1   label       450 non-null    object
 2   value       450 non-null    int64 
 3   reason      448 non-null    object
dtypes: int64(1), object(3)
memory usage: 17.6+ KB


In [ ]:
concatenated_pred__df.head()

,post_title,label,value,reason
0,I donât know if Iâm depressed.,Feeling bad about yourself or that you are a f...,1,The given label mentions feeling bad about one...
1,I donât know if Iâm depressed.,Feeling down depressed or hopeless,1,The post text mentions feeling terrible lately...
2,I donât know if Iâm depressed.,Feeling tired or having little energy,1,"The post text mentions a loss of appetite, lac..."
3,I donât know if Iâm depressed.,Little interest or pleasure in doing,1,"The post text mentions a loss of appetite, fee..."
4,I donât know if Iâm depressed.,Moving or speaking so slowly that other people...,0,The given label mentions a symptom related to ...


# Evaluation Metrices for the second 450 records
(actually second 50 in the main csv file)

In [ ]:
true_values = concatenated_true__df['value'].values
predicted_values = concatenated_pred__df['value'].values

# Calculate accuracy
accuracy = accuracy_score(true_values, predicted_values)
print(f" ------------------------ The Accuracy of GPT3.5 is: {accuracy} ------------------------")
# Calculate precision
precision = precision_score(true_values, predicted_values)
print(f" ------------------------ The Precision of GPT3.5 is: {precision} ------------------------")

# Calculate recall
recall = recall_score(true_values, predicted_values)
print(f" ------------------------ The Recall of GPT3.5 is: {recall} ------------------------")

# Calculate F1-score
f1 = f1_score(true_values, predicted_values)
print(f" ------------------------ The F1-Score of GPT3.5 is: {f1} ------------------------")

ValueError: ignored

In [ ]:
true_values

In [ ]:
import numpy as np
true_values = np.asarray(true_values, dtype = 'int')

In [ ]:
predicted_values

array([1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,

# Final Check for 200(1800) of data

## True 50(450) to 250(2250)

In [56]:
true_50_to_250_df = primate_data_single_label[450:2250]
true_50_to_250_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 450 to 2249
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  1800 non-null   object
 1   post_text   1800 non-null   object
 2   label       1800 non-null   object
 3   value       1800 non-null   object
dtypes: object(4)
memory usage: 56.4+ KB


In [77]:
true_50_to_250_df.head(-20)

,post_title,post_text,label,value
450,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Feeling bad about yourself or that you are a f...,1
451,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Feeling down depressed or hopeless,1.0
452,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Feeling tired or having little energy,1
453,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Little interest or pleasure in doing,1
454,I don’t know if I’m depressed.,"I’m 17, I don’t know how I should word this be...",Moving or speaking so slowly that other people...,0
...,...,...,...,...
2225,I'm so useless that my psychiatrist doesn't wa...,I have this horrible self esteem issues and I'...,Feeling tired or having little energy,1
2226,I'm so useless that my psychiatrist doesn't wa...,I have this horrible self esteem issues and I'...,Little interest or pleasure in doing,1
2227,I'm so useless that my psychiatrist doesn't wa...,I have this horrible self esteem issues and I'...,Moving or speaking so slowly that other people...,1
2228,I'm so useless that my psychiatrist doesn't wa...,I have this horrible self esteem issues and I'...,Poor appetite or overeating,0.0


In [84]:
true_50_to_250_df['value'].unique()
# true_50_to_250_df['value'] = true_50_to_250_df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1, 'No': 0, 'unknown': 0})

array([1, 0])

In [83]:
true_50_to_250_df["value"] = true_50_to_250_df["value"].astype(int)

<ipython-input-83-64acbebfe785>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_50_to_250_df["value"] = true_50_to_250_df["value"].astype(int)


## Pred 50 to 250

In [ ]:
pred_50_to_60_df = pd.read_csv(f"50_to_60_data_preprocessed.csv",  encoding='latin-1')
pred_50_to_60_df.info()

In [62]:
pred_60_to_80_df = pd.read_csv(f"60_to_80_data_preprocessed.csv",  encoding='latin-1')
pred_60_to_80_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  180 non-null    object
 1   label       180 non-null    object
 2   value       180 non-null    object
 3   reason      180 non-null    object
dtypes: object(4)
memory usage: 5.8+ KB


In [ ]:
pred_80_to_90_df = pd.read_csv(f"80_to_90_data_preprocessed.csv",  encoding='latin-1')
pred_80_to_90_df.info()

In [ ]:
pred_90_to_100_df = pd.read_csv(f"90_to_100_data_preprocessed.csv",  encoding='latin-1')
pred_90_to_100_df.info()

In [65]:
pred_100_to_110_df = pd.read_csv(f"100_to_110_data_checked.csv",  encoding='latin-1')
pred_100_to_110_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label       89 non-null     object
 2   value       90 non-null     object
 3   reason      87 non-null     object
dtypes: object(4)
memory usage: 2.9+ KB


In [66]:
pred_110_to_150_df = pd.read_csv(f"110_to_150_data_checked.csv",  encoding='latin-1')
pred_110_to_150_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  360 non-null    object
 1   label       360 non-null    object
 2   value       360 non-null    object
 3   reason      357 non-null    object
dtypes: object(4)
memory usage: 11.4+ KB


In [67]:
pred_150_to_180_df = pd.read_csv(f"150_to_180_data_checked.csv",  encoding='latin-1')
pred_150_to_180_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  270 non-null    object
 1   label       270 non-null    object
 2   value       270 non-null    object
 3   reason      269 non-null    object
dtypes: object(4)
memory usage: 8.6+ KB


In [ ]:
pred_180_to_200_df = pd.read_csv(f"180_to_200_data_checked.csv",  encoding='latin-1')
pred_180_to_200_df.info()

In [ ]:
pred_200_to_225_df = pd.read_csv(f"200_to_225_data_chekced.csv",  encoding='latin-1')
pred_200_to_225_df.info()

In [ ]:
pred_225_to_250_df = pd.read_csv(f"225_to_250_data_checked.csv",  encoding='latin-1')
pred_225_to_250_df.info()

In [74]:
concatenated_pred__df = pd.concat([pred_50_to_60_df,
                                   pred_60_to_80_df,
                                   pred_80_to_90_df,
                                   pred_90_to_100_df,
                                   pred_100_to_110_df,
                                   pred_110_to_150_df,
                                   pred_150_to_180_df,
                                   pred_180_to_200_df,
                                   pred_200_to_225_df,
                                   pred_225_to_250_df])

In [75]:
concatenated_pred__df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1800 entries, 0 to 224
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  1800 non-null   object
 1   label       1799 non-null   object
 2   value       1800 non-null   object
 3   reason      1781 non-null   object
dtypes: object(4)
memory usage: 70.3+ KB


In [78]:
concatenated_pred__df.head(-20)

,post_title,label,value,reason
0,I donât know if Iâm depressed.,Feeling bad about yourself or that you are a f...,yes,The given label mentions feeling bad about one...
1,I donât know if Iâm depressed.,Feeling down depressed or hopeless,yes,The post text mentions feeling terrible lately...
2,I donât know if Iâm depressed.,Feeling tired or having little energy,yes,"The post text mentions a loss of appetite, lac..."
3,I donât know if Iâm depressed.,Little interest or pleasure in doing,yes,"The post text mentions a loss of appetite, fee..."
4,I donât know if Iâm depressed.,Moving or speaking so slowly that other people...,no,The given label mentions a symptom related to ...
...,...,...,...,...
200,I'm so useless that my psychiatrist doesn't wa...,Feeling tired or having little energy,yes,"The post text mentions feeling tired, lack of ..."
201,I'm so useless that my psychiatrist doesn't wa...,Little interest or pleasure in doing,yes,The post text indicates that the individual is...
202,I'm so useless that my psychiatrist doesn't wa...,Moving or speaking so slowly that other people...,no,The given label is not mentioned in the post c...
203,I'm so useless that my psychiatrist doesn't wa...,Poor appetite or overeating,no,The post text mentions the person's struggle w...


In [81]:
concatenated_pred__df['value'].unique()

array([1, 0])

In [80]:
concatenated_pred__df['value'] = concatenated_pred__df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1, 'No': 0, 'unknown': 0})

In [85]:
true_values = true_50_to_250_df['value'].values
predicted_values = concatenated_pred__df['value'].values

# Calculate accuracy
accuracy = accuracy_score(true_values, predicted_values)
print(f" ------------------------ The Accuracy of GPT3.5 is: {accuracy} ------------------------")
# Calculate precision
precision = precision_score(true_values, predicted_values)
print(f" ------------------------ The Precision of GPT3.5 is: {precision} ------------------------")

# Calculate recall
recall = recall_score(true_values, predicted_values)
print(f" ------------------------ The Recall of GPT3.5 is: {recall} ------------------------")

# Calculate F1-score
f1 = f1_score(true_values, predicted_values)
print(f" ------------------------ The F1-Score of GPT3.5 is: {f1} ------------------------")

 ------------------------ The Accuracy of GPT3.5 is: 0.7222222222222222 ------------------------
 ------------------------ The Precision of GPT3.5 is: 0.6147186147186147 ------------------------
 ------------------------ The Recall of GPT3.5 is: 0.797752808988764 ------------------------
 ------------------------ The F1-Score of GPT3.5 is: 0.6943765281173594 ------------------------


In [86]:
true_50_to_250_df[456:798]

,post_title,post_text,label,value
906,"Please help, I'm so alone","I am so alone. Not just lonely, completely alo...",Thoughts that you would be better off dead or ...,0
907,"Please help, I'm so alone","I am so alone. Not just lonely, completely alo...",Trouble concentrating on things such as readin...,0
908,"Please help, I'm so alone","I am so alone. Not just lonely, completely alo...",Trouble falling or staying asleep or sleeping ...,0
909,What to expect when you share your mental heal...,I recently shared some parts of my mental heal...,Feeling bad about yourself or that you are a f...,1
910,What to expect when you share your mental heal...,I recently shared some parts of my mental heal...,Feeling down depressed or hopeless,1
...,...,...,...,...
1243,I feel like my only option to get out of going...,My depression has gotten so bad that the small...,Feeling down depressed or hopeless,1
1244,I feel like my only option to get out of going...,My depression has gotten so bad that the small...,Feeling tired or having little energy,1
1245,I feel like my only option to get out of going...,My depression has gotten so bad that the small...,Little interest or pleasure in doing,0
1246,I feel like my only option to get out of going...,My depression has gotten so bad that the small...,Moving or speaking so slowly that other people...,0


In [87]:
concatenated_pred__df[456:798]

,post_title,label,value,reason
6,"Please help, I'm so alone",Thoughts that you would be better off dead or ...,0,The given label refers to thoughts of self-har...
7,"Please help, I'm so alone",Trouble concentrating on things such as readin...,0,The given label is unrelated to the content of...
8,"Please help, I'm so alone",Trouble falling or staying asleep or sleeping ...,0,The post text does not mention anything about ...
9,What to expect when you share your mental heal...,Feeling bad about yourself or that you are a f...,1,The post text reflects the author's concern ab...
10,What to expect when you share your mental heal...,Feeling down depressed or hopeless,1,The post text mentions the author's experience...
...,...,...,...,...
253,I feel like my only option to get out of going...,Feeling down depressed or hopeless,1,The post text indicates severe depressive symp...
254,I feel like my only option to get out of going...,Feeling tired or having little energy,1,The post text describes the person's extreme e...
255,I feel like my only option to get out of going...,Little interest or pleasure in doing,1,The post text indicates that the person is exp...
256,I feel like my only option to get out of going...,Moving or speaking so slowly that other people...,1,The given label is related to the symptoms of ...
